# Efficient Quantum Kernel Evaluation Demo
In this notebook we create a symmetric fidelity quantum kernel where every entry is calculated by a 2 qubits circuit. We run our code both on a simulator and on a real QPU to demonstrate our Efficient Quantum Kernel Evaluation implementation.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, SamplerV2
import numpy as np
from qiskit.circuit.library import ZZFeatureMap
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager   
from qiskit_machine_learning.state_fidelities import ComputeUncompute    


In [2]:
#we import our efficient implemetation of the FidelityQuantumKernel which is compatible with the qiskit-machine-learning implementation

from fidelity_quantum_kernel_efficient import FidelityQuantumKernel 

We now create a dataset which will be used to to evaluate the kernel

In [3]:
num_features = 2         #this is the dimensionality of every point in the dataset 
#we map every feature to a qubit in the fidelity quantum circuit used to compute each kernel entry. 
#Therefore, num_features is also the number of qubits used by the fidelity circuit.
number_of_points_X = 8 #8
number_of_points_Y = 10 #10

#create data
np.random.seed(seed=123)
X = np.random.uniform(-np.pi/2, np.pi/2, size = (number_of_points_X,num_features))
Y = np.random.uniform(-np.pi/2, np.pi/2, size = (number_of_points_Y,num_features))

We coonect to the qiskit runtime service to both a simulator and a real QPU. We then create a feature map which will be used to compute the fidelity and create an instance 

In [4]:
service = QiskitRuntimeService()

#connection to a real backend
backend_QPU = service.least_busy(simulator=False, operational=True)  # to select the least busy quantum machine available
num_qubits_backend_QPU = backend_QPU.num_qubits
session_QPU = Session(backend=backend_QPU)
pass_manager_QPU = generate_preset_pass_manager(optimization_level=1, backend=backend_QPU)
sampler_QPU = SamplerV2(mode=session_QPU)

#connection to a simulator
num_qubits_backend_sim = 10
backend_sim = GenericBackendV2(num_qubits=num_qubits_backend_sim, seed = 1)                         
session_sim = Session(backend=backend_sim)
pass_manager_sim = generate_preset_pass_manager(optimization_level=1, backend=backend_sim)
sampler_sim = SamplerV2(mode=session_sim)


#define a feature map 
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1, entanglement='linear')

In order to evaluate the kernel on the data one simply needs to call the .evaluate method (identical to the qiskit-machine-learning implementation) or the .evaluate_efficient method, which will parallelize the fidelity circuits runs when running on a real QPU. 

In [5]:
# CASE 1, efficient implementation on a simulator

fidelity_efficient_sim = ComputeUncompute(sampler=sampler_sim)
qkernel_efficient_sim = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_efficient_sim)

#simmetric kernel matrix
#kernel_matrix_efficient_sim = qkernel_efficient_sim.evaluate_efficient(X, backend = backend_sim, pass_manager = pass_manager_sim, shots = 4096)

#non simmtric kernel matrix
kernel_matrix_efficient_sim = qkernel_efficient_sim.evaluate_efficient(X, y_vec= Y,  backend = backend_sim, pass_manager = pass_manager_sim, shots = 4096)


print(kernel_matrix_efficient_sim)


c:\Users\melch\.conda\envs\qpu_demo\Lib\site-packages\qiskit\providers\fake_provider\generic_backend_v2.py:506: RuntimeWarning: Aer not found using BasicSimulator and no noise
  warnings.warn("Aer not found using BasicSimulator and no noise", RuntimeWarning)


[[0.10888672 0.35595703 0.03735352 0.24487305 0.67431641 0.02490234
  0.31835938 0.16040039 0.21337891 0.40893555]
 [0.04858398 0.26147461 0.03222656 0.02783203 0.02856445 0.17944336
  0.00341797 0.67773438 0.0534668  0.27978516]
 [0.12207031 0.23095703 0.0534668  0.00366211 0.50415039 0.14160156
  0.27636719 0.14819336 0.18920898 0.11523438]
 [0.07348633 0.18164062 0.1940918  0.3527832  0.44775391 0.33886719
  0.29736328 0.10522461 0.02246094 0.58105469]
 [0.21606445 0.08276367 0.08862305 0.2121582  0.15258789 0.65576172
  0.24633789 0.08789062 0.65991211 0.01831055]
 [0.16943359 0.37890625 0.30834961 0.11157227 0.05834961 0.00268555
  0.00366211 0.18066406 0.06274414 0.06274414]
 [0.17236328 0.01855469 0.41503906 0.49438477 0.18432617 0.35180664
  0.18066406 0.07714844 0.00195312 0.31542969]
 [0.09619141 0.20922852 0.41503906 0.10424805 0.32714844 0.00170898
  0.16943359 0.12695312 0.24682617 0.01196289]]


In [6]:
# CASE 2, efficient implementation on a QPU

fidelity_efficient_QPU = ComputeUncompute(sampler=sampler_QPU)
qkernel_efficient_QPU = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_efficient_QPU)
kernel_matrix_efficient_QPU = qkernel_efficient_QPU.evaluate_efficient(X, y_vec= Y,  backend = backend_QPU, pass_manager = pass_manager_QPU, shots = 4096)

print(kernel_matrix_efficient_QPU)  

#NOTE: this run used 4 seconds and 2 jobs on  ibm_kyiv

[[0.12939453 0.24853516 0.12768555 0.33032227 0.14404297 0.02294922
  0.13867188 0.2800293  0.36474609 0.25561523]
 [0.30883789 0.07714844 0.29589844 0.57397461 0.04760742 0.31787109
  0.1472168  0.26318359 0.25561523 0.1081543 ]
 [0.05224609 0.0847168  0.08276367 0.18457031 0.01831055 0.60961914
  0.09008789 0.1394043  0.09790039 0.23974609]
 [0.05664062 0.53271484 0.13012695 0.28833008 0.16577148 0.19067383
  0.22949219 0.1027832  0.21386719 0.39038086]
 [0.45458984 0.31396484 0.28149414 0.1105957  0.06958008 0.47387695
  0.19970703 0.10180664 0.14111328 0.19482422]
 [0.58300781 0.23095703 0.08325195 0.63647461 0.0690918  0.20751953
  0.3894043  0.29760742 0.11352539 0.06225586]
 [0.02709961 0.25830078 0.08129883 0.47924805 0.18969727 0.41381836
  0.12036133 0.29931641 0.0378418  0.12573242]
 [0.12133789 0.20556641 0.05908203 0.21191406 0.36425781 0.19946289
  0.0793457  0.03149414 0.17456055 0.10742188]]


In [7]:
# CASE 3, standard implementation on a simulator


fidelity_sim = ComputeUncompute(sampler=sampler_sim,  pass_manager=pass_manager_sim)
qkernel_sim = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_sim)
kernel_matrix_sim = qkernel_sim.evaluate(X, Y)

print(kernel_matrix_sim)

c:\Users\melch\.conda\envs\qpu_demo\Lib\site-packages\qiskit\providers\fake_provider\generic_backend_v2.py:506: RuntimeWarning: Aer not found using BasicSimulator and no noise
  warnings.warn("Aer not found using BasicSimulator and no noise", RuntimeWarning)


[[0.09082031 0.35058594 0.04101562 0.22851562 0.67871094 0.02539062
  0.32421875 0.15136719 0.22363281 0.40429688]
 [0.04394531 0.25       0.0390625  0.03027344 0.03125    0.16503906
  0.00097656 0.66992188 0.07128906 0.26953125]
 [0.125      0.21386719 0.05859375 0.00488281 0.4921875  0.13964844
  0.26660156 0.15136719 0.19238281 0.13476562]
 [0.078125   0.17285156 0.19335938 0.33691406 0.42675781 0.33789062
  0.2890625  0.1171875  0.01367188 0.55371094]
 [0.23144531 0.08984375 0.11035156 0.22167969 0.16601562 0.66308594
  0.24121094 0.07910156 0.66308594 0.02050781]
 [0.16796875 0.42871094 0.30859375 0.11035156 0.05175781 0.
  0.00488281 0.17871094 0.04785156 0.06933594]
 [0.16308594 0.02050781 0.40917969 0.50292969 0.1796875  0.32714844
  0.18554688 0.08007812 0.00292969 0.32421875]
 [0.10058594 0.20800781 0.40527344 0.11816406 0.33203125 0.00292969
  0.21582031 0.125      0.23144531 0.01074219]]


In [8]:
# CASE 4, standard implementation on a QPU

fidelity_QPU = ComputeUncompute(sampler=sampler_QPU,  pass_manager=pass_manager_QPU)
qkernel_QPU = FidelityQuantumKernel(feature_map = feature_map, fidelity = fidelity_QPU)
kernel_matrix_QPU = qkernel_QPU.evaluate(X, Y)

print(kernel_matrix_QPU)

#NOTE: this run used 87 seconds and 80 jobs on  ibm_kyiv

[[0.11694336 0.35888672 0.04003906 0.24755859 0.64331055 0.03808594
  0.31591797 0.16845703 0.22583008 0.38769531]
 [0.06518555 0.26000977 0.03662109 0.03735352 0.03588867 0.18920898
  0.00854492 0.66235352 0.0637207  0.26318359]
 [0.11572266 0.22119141 0.07104492 0.01123047 0.47583008 0.13989258
  0.27832031 0.15429688 0.18041992 0.12695312]
 [0.0859375  0.19873047 0.18432617 0.36865234 0.42333984 0.32666016
  0.27197266 0.10473633 0.02905273 0.56225586]
 [0.21386719 0.08789062 0.11157227 0.22338867 0.14599609 0.6237793
  0.24291992 0.09960938 0.64135742 0.01831055]
 [0.16381836 0.37939453 0.28881836 0.12304688 0.05029297 0.0144043
  0.00952148 0.18994141 0.07128906 0.06958008]
 [0.16894531 0.04272461 0.38256836 0.49829102 0.19433594 0.34399414
  0.17675781 0.07910156 0.01171875 0.31103516]
 [0.10424805 0.19506836 0.38330078 0.1262207  0.3215332  0.01879883
  0.16333008 0.14794922 0.26147461 0.02270508]]
